# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import findspark
import gzip
import json
import dataExtractUtils as de
import uuid
import constants

findspark.init()

from pyspark.sql import SparkSession

#Locate spark on local machine

print(dir(de))

print(pd.get_option("display.max_colwidth"))
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows",None)
print(pd.get_option("display.max_rows"))

['NA', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'constants', 'country_code_lookup', 'datetime', 'keyname', 'matchCountriesInAffiliation', 'processArticle', 'processAuthor', 'processPublishedDate', 'uuid', 'year_key_lookup']
50
None


### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [ ]:
#Setup SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
# Read in the data here
#Test file path /home/jswainston/Downloads/0.json.gz
#df = spark.read.json("/home/jswainston/Downloads/April2022CrossrefPublicDataFile/0.json.gz",multiLine=True)
#df.printSchema()
#df.show()


with gzip.open("/home/jswainston/Downloads/April2022CrossrefPublicDataFile/0.json.gz", 'r') as fin:        
    json_bytes = fin.read()                      

json_str = json_bytes.decode('utf-8')            
data = json.loads(json_str)                      
print("data object type is " + str(type(data)))

print("Data has the following keys " + str(data.keys()))
print("The type of the value corresponding to the item key is " + str(type(data["items"])))
print("The length of the list contained in the items dict is " + str(len(data["items"])))
print(type(data["items"][0]))
print(len((data["items"][0])))
print(data["items"][0].keys())
print(data["items"][0].keys())
print("date part type test " + str(type(data["items"][0]["published"]["date-parts"])))
print(type(data["items"][0]["published"]["date-parts"][0]))
print(type(data["items"][0]["published"]["date-parts"][0][0]))
print(data["items"][0]["published"]["date-parts"][0][0])

#access author affiliations
#for item in data["items"]:
#    if "author" in item:
#        for author in item["author"]:
#            for affiliation in author["affiliation"]:
#                print(affiliation["name"]) 





data object type is <class 'dict'>
Data has the following keys dict_keys(['items'])
The type of the value corresponding to the item key is <class 'list'>
The length of the list contained in the items dict is 5000
<class 'dict'>
33
dict_keys(['URL', 'resource', 'member', 'score', 'created', 'license', 'ISSN', 'container-title', 'issued', 'issue', 'prefix', 'reference-count', 'indexed', 'author', 'DOI', 'is-referenced-by-count', 'published', 'published-print', 'subject', 'published-online', 'content-domain', 'title', 'link', 'source', 'type', 'publisher', 'journal-issue', 'volume', 'references-count', 'issn-type', 'deposited', 'page', 'short-container-title'])
dict_keys(['URL', 'resource', 'member', 'score', 'created', 'license', 'ISSN', 'container-title', 'issued', 'issue', 'prefix', 'reference-count', 'indexed', 'author', 'DOI', 'is-referenced-by-count', 'published', 'published-print', 'subject', 'published-online', 'content-domain', 'title', 'link', 'source', 'type', 'publisher', 'jou

In [4]:
# test retreiving country values from affiliations
i=0
for item in data["items"]:
    print("DOI " + item["DOI"])
    if "author" in item:
        for author in item["author"]:
            i = i + 1
            print("Author number " + str(i))
            print(uuid.uuid4())
            for affiliation in author["affiliation"]:
                affiliationCountries = de.matchCountriesInAffiliation(affiliation["name"])
                print(affiliationCountries) 

DOI 10.1149/1.1392467
Author number 1
2fb3fe56-2329-4d24-81bb-ee6e9a4b5ef0
['Italy']
Author number 2
95ae142b-c72e-49ec-a3f9-dcbe917b27f7
['Italy']
Author number 3
87801dd5-b3d3-42f0-85be-cc351c927ac7
['Italy']
Author number 4
5a3cbf3a-e052-4dab-b4a5-69f75efd19cb
['Italy']
DOI 10.1108/09564239810199969
Author number 5
12fdc98f-9b32-43dc-b907-61499b5de353
DOI 10.1108/08880450010310462
Author number 6
4f1d88c5-19ac-4704-b2c0-6876103d3097
DOI 10.1134/1.1493384
Author number 7
247c578e-f3a9-4645-9e62-cd95c81aeb13
Author number 8
ca683484-3967-446b-884b-dfc23af76a2c
Author number 9
843bac32-fd8e-4afc-8b06-9688735d7843
DOI 10.1128/cdli.9.4.753-762.2002
Author number 10
e9484b63-4452-4d69-a3e2-e9628618475d
['France']
DOI 10.1108/01437729710169300
Author number 11
aaea158c-68e2-4523-aa18-c96daac7d825
DOI 10.1016/s8755-4615(97)90024-4
Author number 12
e8e5d2f6-c3bb-41eb-afbf-1f1c96567543
DOI 10.1108/09578230010378377
Author number 13
f43cf43c-9121-48a3-9b11-680b402e7a9c
Author number 14
97a6659

In [3]:
#Extract data into DataFrames for each entity we are inerested in

articleData = {
    'article_id' : [],
    'DOI' : [],
    'title' : [],
    'published_date' : []
}

authorData = {
    'author_id' : [],
    'article_id' : [],
    'first_name' : [],
    'last_name' : []
}

affiliationData = {
    'affiliation_id' : [],
    'author_id' : [],
    'country_name' : []
}

affiliationCountry = {
    'country_ID' : [],
    'affiliation_id' : []
}

affiliationErrors = {
    'affiliation_id' : [],
    'author_id' : [],
    'affiliation_name' : []
}

for item in data["items"]:
    if "type" in item:
        if item["type"] == 'journal-article':
            article = de.processArticle(item)
            articleData['article_id'].append(article[0])
            articleData['DOI'].append(article[1])
            articleData['title'].append(article[2])
            articleData['published_date'].append(article[3])

            authors = de.processAuthor(item,article[0]) # pass article id in so it can be used as foreign key for author
            affiliationErrorData = authors[2]
            authorAffiliations = authors[1]
            authors = authors[0]

            
            for author in authors:
                authorData['author_id'].append(author[0])
                authorData['article_id'].append(author[1])
                authorData['first_name'].append(author[2])
                authorData['last_name'].append(author[3])

            if authorAffiliations is not None:
                for affiliation in authorAffiliations:
                    affiliationData['affiliation_id'].append(affiliation[0])
                    affiliationData['author_id'].append(affiliation[1])
                    affiliationData['country_name'].append(affiliation[2])

            if affiliationErrorData is not None:
                for affiliationError in affiliationErrorData:
                    affiliationErrors['affiliation_id'].append(affiliationError[0])
                    affiliationErrors['author_id'].append(affiliationError[1])
                    affiliationErrors['affiliation_name'].append(affiliationError[2])

            

articles = pd.DataFrame(articleData)
authors = pd.DataFrame(authorData)
affiliations = pd.DataFrame(affiliationData)
affiliationErrors = pd.DataFrame(affiliationErrors)



In [4]:
#tuple(articles.to_numpy().tolist())
#articles.values[0].tolist()
#articles[articles['published_date'].isnull()]
articles[articles['DOI'] == "10.1149/1.1394094"]
#articles.where(pd.notnull(articles), None)
#articles[articles.isna().any(axis=1)]

,article_id,DOI,title,published_date
172,309182e5-164f-4fd3-b73d-bfb623bb43c1,10.1149/1.1394094,Observation and Characterization of Aluminum Passivation in Silicon-Doped KOH Solutions,2000-01-01


In [3]:
# show articles sample
articles.head()

NameError: name 'articles' is not defined

In [5]:
# show authors sample
authors.head()

,author_id,article_id,first_name,last_name
0,55443a81-71aa-4e2f-8300-c1484871cdf7,197f055d-abaf-4391-90dc-4d38a28da160,Carlo,Cavallotti
1,e87ab4d3-bfdb-4996-8b06-ad87ba932b10,197f055d-abaf-4391-90dc-4d38a28da160,Valeria,Bertani
2,142ee9ff-8dc8-486c-8963-7b69544ee796,197f055d-abaf-4391-90dc-4d38a28da160,Maurizio,Masi
3,c6962572-8f70-4a0e-a662-58e8a6703640,197f055d-abaf-4391-90dc-4d38a28da160,Sergio,Carrà
4,91a9f6f1-5a3c-4a43-b573-ed132d5d9a9c,1a4e6fea-25d0-46dd-b5f4-52f9d2cd91b4,Audrey,Gilmore


In [6]:
# show affiliation sample
affiliations.head()

,affiliation_id,author_id,country_name
0,ba2e0ef7-6479-44b3-91b7-bee1e22551e5,55443a81-71aa-4e2f-8300-c1484871cdf7,Italy
1,c56abd3a-2a37-4faf-bac6-755dcce41399,e87ab4d3-bfdb-4996-8b06-ad87ba932b10,Italy
2,2d1d1f01-89d0-4441-ad06-d8afeea5076e,142ee9ff-8dc8-486c-8963-7b69544ee796,Italy
3,3dc84878-e6c4-4085-8a9a-7fc44acba4ac,c6962572-8f70-4a0e-a662-58e8a6703640,Italy
4,1a2a3ce0-d92c-4122-b9fb-3dc46df2b67c,c358f482-a9d5-4c24-8b70-0f6f9973fbd9,France


In [22]:
#show affiliations where a country wasn't found
affiliationErrors["affiliation_name"]

0                                                                                                                                                                                                                                                                                                                                                                                                                                                  Department of Chemistry, University of California, Irvine, California 92697-2025
1                                                                                                                                                                                                                                                                                                                                                                                                                                                  Department of Chemistry, University of California

In [16]:
worldbank_path = '../data/WorldBank/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_4481247.csv'
gdp_data = pd.read_csv(worldbank_path,header=2)

#gdp_data[]

gdp_data[['Country Name','Country Code','Indicator Name','2015','2016','2017','2018','2019','2020']].head()

#i = 0
#j = 0
#for code in gdp_data['Country Code']:
#    if code in constants.COUNTRIES.values():
#        i+=1;
#        #print(code + " has a match in countries")
#    else:
#        j+=1;
#        print (code + " doesn't have a match in countries")


#print("total matches = " + str(i))
#print("total not matching = " + str(j))

# year range from article data is 1834 to 2116. Create year dimension from 1800 to 2200
#select distinct date_part('year',published_date) as year from articles order by year

,Country Name,Country Code,Indicator Name,2015,2016,2017,2018,2019,2020
0,Aruba,ABW,GDP (current US$),2.963128e+09,2.983799e+09,3.092179e+09,3.202235e+09,3.310056e+09,2.496648e+09
1,Africa Eastern and Southern,AFE,GDP (current US$),9.242525e+11,8.823551e+11,1.020647e+12,9.910223e+11,9.975340e+11,9.216459e+11
2,Afghanistan,AFG,GDP (current US$),1.913421e+10,1.811656e+10,1.875347e+10,1.805323e+10,1.879945e+10,2.011614e+10
3,Africa Western and Central,AFW,GDP (current US$),7.607345e+11,6.905464e+11,6.837487e+11,7.416899e+11,7.945430e+11,7.844457e+11
4,Angola,AGO,GDP (current US$),8.721929e+10,4.984049e+10,6.897276e+10,7.779294e+10,6.930910e+10,5.361907e+10


In [18]:
dim_year_data = {
    'year_id' : list(range(1,402)),
    'year' : list(range(1800,2201))
}

#year_id = list(range(1,402))
#year = list(range(1800,2201))

dim_year = pd.DataFrame(dim_year_data)

dim_year.head()

,year_id,year
0,1,1800
1,2,1801
2,3,1802
3,4,1803
4,5,1804


In [13]:
country_data = {
    'country_code' : [],
    'country_name' : []
}

for code in constants.COUNTRIES.values():
    country_data['country_code'].append(code)

for name in constants.COUNTRIES.keys():
    country_data['country_name'].append(name)

countrydf = pd.DataFrame(country_data)
countrydf = countrydf.dropna()
countrydf = countrydf.drop_duplicates(subset=['country_code'])
countrydf = countrydf.sort_values('country_code')
countrydf.loc[countrydf['country_code'] == 'GBR', 'country_name'] = 'United Kingdom'
countrydf.loc[countrydf['country_code'] == 'AUS', 'country_name'] = 'Australia'
countrydf.loc[countrydf['country_code'] == 'FRA', 'country_name'] = 'France'
countrydf = countrydf.reset_index(drop=True)
countrydf['country_id'] = countrydf.index
countrydf[['country_id','country_code','country_name']]




,country_id,country_code,country_name
0,0,ABW,Aruba
1,1,AFG,Afghanistan
2,2,AGO,Angola
3,3,AIA,Anguilla
4,4,ALB,Albania
5,5,AND,Andorra
6,6,ARE,United Arab Emirates
7,7,ARG,Argentina
8,8,ARM,Armenia
9,9,ASM,American Samoa


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here





### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.